# Advanced Heiken Ashi Strategy: Trend Confirmation & Fib Exits
This notebook implements a sophisticated trading strategy combining:
1.  **Trend Confirmation**: Using Market Structure Shift (MSS/BOS) to ensure trades only occur in confirmed upward trends.
2.  **Entry Signal**: First Green Heiken Ashi candle after trend confirmation.
3.  **Exit Strategy**: Sell only if the candle is Red **AND** price has broken below a key Fibonacci retracement level (61.8%).


## 0) Setup


In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show

# --- Setup correct working directory (ROOT) ---
if os.getcwd().endswith('notebooks'):
    os.chdir('..')

ROOT = os.getcwd()
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

from src.backtester.data import load_cleaned_assets, align_close_prices
from src.backtester.engine import BacktestConfig, run_backtest
from src.backtester.metrics import compute_performance_stats
from src.backtester.models import TrendHeikenAshiFibModel
from src.backtester.bokeh_plots import build_interactive_portfolio_layout
from src.backtester.report import compute_backtest_report

output_notebook()


Loading BokehJS ...

## 1) Data Loading


In [2]:
assets = load_cleaned_assets(symbols=None)
close = align_close_prices(assets)

cfg = BacktestConfig(
    initial_equity=1_000_000,
    transaction_cost_bps=5,
    mode='event_driven',
    strict_signals=True, # Discrete entry/exit
    stop_loss_pct=0.0
)

model = TrendHeikenAshiFibModel(fib_level='fib_61_8', lookback_window=60)
print(f'Loaded {len(assets)} assets.')


Loaded 100 assets.


## 2) Backtest Execution


In [3]:
print('Computing Signals...')
signals = model.compute_signals(assets)

print('Running Backtest...')
res = run_backtest(close_prices=close, weights=signals, config=cfg)
report = compute_backtest_report(result=res, close_prices=close)
display(report)


Computing Signals...
Running Backtest...


Start                         2016-01-25 00:00:00
End                           2026-01-16 00:00:00
Duration                       3644 days 00:00:00
Initial Equity                          1000000.0
Final Equity                       1879313.237597
Equity Peak                        3294251.077123
Total Return [%]                        87.931324
CAGR [%]                                 6.539114
Volatility (ann) [%]                    28.287175
Sharpe                                   0.363163
Sortino                                  0.597994
Max Drawdown [%]                       -59.234152
Calmar                                   0.110394
Best Day [%]                            29.787117
Worst Day [%]                           -13.03212
Avg Gross Exposure                       0.987655
Avg Net Exposure                         0.987655
Exposure Time [%]                       99.362804
Rebalance Days                                499
Total Turnover                   225685387.518696


## 3) Interactive Visualization


In [4]:
def build_market_proxy_ohlcv(assets, index):
    opens = pd.concat([df['Open'].reindex(index).astype(float) for df in assets.values()], axis=1).mean(axis=1)
    highs = pd.concat([df['High'].reindex(index).astype(float) for df in assets.values()], axis=1).mean(axis=1)
    lows = pd.concat([df['Low'].reindex(index).astype(float) for df in assets.values()], axis=1).mean(axis=1)
    closes = pd.concat([df['Close'].reindex(index).astype(float) for df in assets.values()], axis=1).mean(axis=1)
    return pd.DataFrame({'Open': opens, 'High': highs, 'Low': lows, 'Close': closes})

market_df = build_market_proxy_ohlcv(assets, close.index)

layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Heiken Ashi Trend + Fib Strategy'
)
show(layout)
